# PARTIE 3 : Transfer learning

Dans cette partie nous utilisons le célèbre algorithme BERT de Google pour notre problématique.

Ce modèle de traitement du langage naturel a été pré-entraîné sur de grands ensembles de donneés textuelles.

Il convient particulièrement pour la détection de spam.




## Import des données

In [29]:
import pandas as pd
df_clean = pd.read_csv('./src/messages_clean.')

In [30]:
df_clean['message_clean'] = df_clean['message_clean'].fillna('')
df_clean.rename(columns={'message_clean': 'message'}, inplace=True)
df_clean.drop(['full_message'], axis=1, inplace=True)
df_clean.head()

,label,message
0,0,jurong point crazy available bugis n great wor...
1,0,ok lar joke wif u oni
2,1,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun early hor u c
4,0,nah think usf live


In [31]:
df_clean.shape

(5572, 2)

## Import librairies

In [32]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

## Preprocessings

## Distinguer les variables

In [33]:
X = df_clean['message'].values
y = df_clean['label'].values

## Diviser le jeu de données en train set et test set

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Tokeniser les données

In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='tf')
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='tf')

## Mettre les labels au bon format

In [35]:
y_train_binary = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test_binary = tf.keras.utils.to_categorical(y_test, num_classes=2)

## Charger le modèle BERT

In [38]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Fine tuning

Nous utisons les valeurs de fine-tuning ci-dessous.

- Nombre d'époques : 3 à 5
- Taille de lot (batch size) : 32 ou 64
- Taux d'apprentissage (learning rate) : 2e-5 soit 0.00002
- Nombre de couches à dégeler : 10

Il s'agit des valeurs conseillées par la documentation pour un cas de données de taille moyenne.

## Geler toutes les couches de BERT

In [39]:
for layer in model.layers:
    layer.trainable = False

## Définir les couches à dégeler

Nous choisissons ici de dégeler les 10 dernières couches.

In [40]:
for layer in model.bert.encoder.layer[-10:]:
    layer.trainable = True

## Compiler le modèle

In [53]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

model.compile(optimizer='adam' , loss=loss, metrics=[metric])

## Entraînement

In [42]:
model.fit(X_train_tokens, y_train_binary, epochs=3, batch_size=32)

Epoch 1/3
140/140 [==============================] - 1964s 14s/step - loss: 0.7028 - accuracy: 0.4892
Epoch 2/3
140/140 [==============================] - 1720s 12s/step - loss: 0.7033 - accuracy: 0.4921
Epoch 3/3
140/140 [==============================] - 1712s 12s/step - loss: 0.7036 - accuracy: 0.4918


Les scores obtenus sont mauvais : 

- l'accuracy est inférieure à 50%

- le modèle n'arrive pas à converger

- la fonction de coût augmente au lieu de diminuer  
--> ceci peut-être dû aux hyperparamètres mal adaptés. Nous avons pourtant utilisé ceux conseillés par la documentation.

Pistes : 

- modifier les hyperparamètres

- augmenter le nombre d'époques

- modifier l'architecture du modèle  
--> il semble que le modèle "n'apprend pas" : il y a certainement un problème dans la construction du modèle.

## Évaluation du modèle

In [43]:
test_loss, test_accuracy = model.evaluate(X_test_tokens, y_test_binary)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

35/35 [==============================] - 395s 11s/step - loss: 0.7090 - accuracy: 0.4922
Test Loss: 0.7089970707893372, Test Accuracy: 0.4921507239341736


## Prédictions

In [46]:
predictions = model.predict(X_test_tokens)
predicted_labels = tf.argmax(predictions[0], axis=1).numpy()

35/35 [==============================] - 383s 11s/step


## Performance

In [48]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_labels))

              precision    recall  f1-score   support

           0       0.90      0.49      0.64       965
           1       0.16      0.65      0.26       150

    accuracy                           0.51      1115
   macro avg       0.53      0.57      0.45      1115
weighted avg       0.80      0.51      0.59      1115



## Matrice de confusion

In [49]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predicted_labels)
print(conf_matrix)

[[474 491]
 [ 53  97]]
